In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Deep_Learning_Project/SubLSTM_Code_BJP_Comments

/content/drive/MyDrive/Deep_Learning_Project/SubLSTM_Code_BJP_Comments


In [3]:
import numpy as np
import pandas as pd
import h5py
import pickle
from copy import deepcopy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing import sequence
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers import Bidirectional

In [4]:
import re
def token(sentence, remove_vowels=False, remove_repeat=False, minchars=2):
    tokens = []
#   for t in re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\w]+",sentence.lower()):
    for t in re.findall("[a-zA-Z]+",sentence.lower()):

        if len(t)>=minchars:
            if remove_vowels:
                t=removeVovels(t)
            if remove_repeat:
                t=removeRepeat(t)
            tokens.append(t)
    return tokens

VOWELS = ['a', 'e', 'i', 'o', 'u']

def removeRepeat(string):
    return re.sub(r'(.)\1+', r'\1\1', string)     

def removeVovels(string):
    return ''.join([l for l in string.lower() if l not in VOWELS])

if __name__ == '__main__':
    pass

def normalize_matrix(matrix):
    pass

In [5]:
SEPERATOR = '\t'
DATA_COLUMN = 0
LABEL_COLUMN = 1
LABELS = ['0','1'] # 0 -> Negative, 1-> Neutral, 2-> Positive
mapping_char2num = {}
mapping_num2char = {}
MAXLEN = 200

#LSTM Model Parameters
#Embedding
MAX_FEATURES = 0
embedding_size = 128
# Convolution
filter_length = 3
nb_filter = 128
pool_length = 3
# LSTM
lstm_output_size = 128
# Training
batch_size = 128
number_of_epochs = 50
numclasses = 3
test_size = 0.2

In [6]:
#Reads the files and splits data into individual lines
	# f=open('/content/drive/My Drive/SubwordLstm/Data/IIITH_Codemixed.txt','r')
	# lines = f.read().lower()
	# lines = lines.lower().split('\n')[:-1]
  """
	Purpose -> Data I/O
	Input   -> Data file containing sentences and labels along with the global variables
	Output  -> Sentences cleaned up in list of lists format along with the labels as a numpy array
	"""

IndentationError: ignored

In [7]:
f=open('BJP_Comments_Youtube.txt','r', encoding='latin1')
lines = f.read().lower()
lines = lines.lower().split('\n')[:-1]
X_train = []
Y_train = []
#Processes individual lines
try:
  for line in lines:
    # Seperator for the current dataset. Currently '\t'. 
    line = line.split('\t')
    print(line[1])
    tokenized_lines = token(line[1])
    # print(tokenized_lines)
    if line[0] == LABELS[0]:
      print(yes)
except:
  pass

label
1
1
1
1
1
1
1


In [87]:
def parse(seperator,datacol,labelcol,labels):
  # lines = pd.read_csv('BJP_Comments_Youtube.csv', encoding='latin1')
  f=open('BJP_Comments_Youtube.txt','r', encoding='latin1')
  lines = f.read().lower()
  lines = lines.lower().split('\n')[:-1]
  X_train = []
  Y_train = []
  #Processes individual lines
  try:
    for line in lines:
      # Seperator for the current dataset. Currently '\t'. 
      line = line.split(seperator)

      #Token is the function which implements basic preprocessing as mentioned in our paper
      tokenized_lines = token(line[datacol])
		
		  #Creates character lists
      char_list = []
      for words in tokenized_lines:
        for char in words:
          char_list.append(char)
        char_list.append(' ')
      #print(char_list) - Debugs the character list created
      X_train.append(char_list)

      #Appends labels
      if line[labelcol] == labels[0]:
        Y_train.append(0)
      if line[labelcol] == labels[1]:
        Y_train.append(1)
      # if line[labelcol] == labels[2]:
      #   Y_train.append(2)
  
    #Converts Y_train to a numpy array	
    Y_train = np.asarray(Y_train)
    assert(len(X_train) == Y_train.shape[0])
  except:
    pass

  return [X_train,Y_train]


In [60]:
def convert_char2num(mapping_n2c,mapping_c2n,trainwords,maxlen):
	"""
	Purpose -> Convert characters to integers, a unique value for every character
	Input   -> Training data (In list of lists format) along with global variables
	Output  -> Converted training data along with global variables
	"""
	allchars = []
	errors = 0

	#Creates a list of all characters present in the dataset
	for line in trainwords:
		try:
			allchars = set(allchars+line)
			allchars = list(allchars)
		except:
			errors += 1

	#print(errors) #Debugging
	#print(allchars) #Debugging 

	#Creates character dictionaries for the characters
	charno = 0
	for char in allchars:
		mapping_char2num[char] = charno
		mapping_num2char[charno] = char
		charno += 1

	assert(len(allchars)==charno) #Checks

	#Converts the data from characters to numbers using dictionaries 
	X_train = []
	for line in trainwords:
		char_list=[]
		for letter in line:
			char_list.append(mapping_char2num[letter])
		#print(no) -- Debugs the number mappings
		X_train.append(char_list)
	print(mapping_char2num)
	print(mapping_num2char)
	#Pads the X_train to get a uniform vector
	#TODO: Automate the selection instead of manual input
	X_train = sequence.pad_sequences(X_train[:], maxlen=maxlen)
	return [X_train,mapping_num2char,mapping_char2num,charno]

In [61]:
def RNN(X_train,y_train,args):
	"""
	Purpose -> Define and train the proposed LSTM network
	Input   -> Data, Labels and model hyperparameters
	Output  -> Trained LSTM network
	"""
	#Sets the model hyperparameters
	#Embedding hyperparameters
	max_features = args[0]
	maxlen = args[1]
	embedding_size = args[2]
	# Convolution hyperparameters
	filter_length = args[3]
	nb_filter = args[4]
	pool_length = args[5]
	# LSTM hyperparameters
	lstm_output_size = args[6]
	# Training hyperparameters
	batch_size = args[7]
	nb_epoch = args[8]
	numclasses = args[9]
	test_size = args[10] 

	#Format conversion for y_train for compatibility with Keras
	y_train = np_utils.to_categorical(y_train, numclasses) 
	#Train & Validation data splitting
	X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_size, random_state=42)
	
	#Build the sequential model
	# Model Architecture is:
	# Input -> Embedding -> Conv1D+Maxpool1D -> LSTM -> LSTM -> FC-1 -> Softmaxloss
	print('Build model...')
	model = Sequential()
	model.add(Embedding(max_features, embedding_size, input_length=maxlen))
	# model.add(Convolution1D(nb_filter=nb_filter,
	# 						filter_length=filter_length,
							
	# 						activation='relu',
	# 						subsample_length=1))
	# model.add(MaxPooling1D(pool_length=pool_length))
	model.add(LSTM(lstm_output_size, dropout_W=0.2, dropout_U=0.2, return_sequences=True))
	model.add(LSTM(lstm_output_size, dropout_W=0.2, dropout_U=0.2, return_sequences=False))
	model.add(Dense(numclasses))
	model.add(Activation('softmax'))

	# Optimizer is Adamax along with categorical crossentropy loss
	model.compile(loss='categorical_crossentropy',
			  	optimizer='adamax',
			  	metrics=['accuracy'])
	

	print('Train...')
	#Trains model for 50 epochs with shuffling after every epoch for training data and validates on validation data
	model.fit(X_train, y_train, 
			  batch_size=batch_size, 
			  shuffle=True, 
			  nb_epoch=nb_epoch,
			  validation_data=(X_valid, y_valid))
	return model

In [88]:
if __name__ == '__main__':
	"""
	Master function
	"""
	print('Starting RNN Engine...\nModel: Char-level LSTM.\nParsing data files...')
	out = parse(SEPERATOR,DATA_COLUMN,LABEL_COLUMN,LABELS)
	X_train = out[0]
	y_train = out[1]
	print('Parsing complete!')

	print('Creating character dictionaries and format conversion in progess...')


Starting RNN Engine...
Model: Char-level LSTM.
Parsing data files...
Parsing complete!
Creating character dictionaries and format conversion in progess...


In [90]:
out = convert_char2num(mapping_num2char,mapping_char2num,X_train,MAXLEN)
mapping_num2char = out[1]
mapping_char2num = out[2]
MAX_FEATURES = out[3]
X_train = np.asarray(out[0])
y_train = np.asarray(y_train).flatten()
print('Complete!')

print('Splitting data into train and test...')
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
	


IndexError: ignored

In [ ]:
y_train = np_utils.to_categorical(y_train ,numclasses) 

In [ ]:

#Train & Validation data splitting
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_size)

In [ ]:
model = Sequential()
model.add(Embedding(27, embedding_size, input_length=MAXLEN))

In [ ]:
model.add(Convolution1D(filters=128,kernel_size=3,
							
							padding='valid',
							activation='relu',
							dilation_rate=1))
model.add(Convolution1D(filters=128,kernel_size=3,
							
							padding='valid',
							activation='relu',
							dilation_rate=1))

In [ ]:

model.add(MaxPooling1D(pool_size=3))

In [ ]:
model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
# model.add(Dropout=(0.2))
model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=False)))
# model.add(Dropout=(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          3456      
_________________________________________________________________
conv1d (Conv1D)              (None, 198, 128)          49280     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 196, 128)          49280     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 65, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 65, 256)           263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 3)                 7

In [ ]:
model.compile(loss='categorical_crossentropy',
        optimizer='adamax',
        metrics=['accuracy'])

In [ ]:
print('Train...')
#Trains model for 50 epochs with shuffling after every epoch for training data and validates on validation data
model.fit(X_train, y_train, 
      batch_size=batch_size, 
      shuffle=True, 
      epochs=50,
      
      validation_data=(X_valid, y_valid))

Train...
Epoch 1/50
20/20 [==============================] - 46s 2s/step - loss: 1.0063 - accuracy: 0.4988 - val_loss: 0.9912 - val_accuracy: 0.4799
Epoch 2/50
20/20 [==============================] - 44s 2s/step - loss: 0.9750 - accuracy: 0.5085 - val_loss: 0.9871 - val_accuracy: 0.4831
Epoch 3/50
20/20 [==============================] - 44s 2s/step - loss: 0.9698 - accuracy: 0.5197 - val_loss: 0.9820 - val_accuracy: 0.4863
Epoch 4/50
20/20 [==============================] - 44s 2s/step - loss: 0.9646 - accuracy: 0.5113 - val_loss: 0.9764 - val_accuracy: 0.4847
Epoch 5/50
20/20 [==============================] - 44s 2s/step - loss: 0.9512 - accuracy: 0.5238 - val_loss: 0.9827 - val_accuracy: 0.4847
Epoch 6/50
20/20 [==============================] - 44s 2s/step - loss: 0.9403 - accuracy: 0.5274 - val_loss: 0.9641 - val_accuracy: 0.5298
Epoch 7/50
20/20 [==============================] - 44s 2s/step - loss: 0.9189 - accuracy: 0.5504 - val_loss: 0.9636 - val_accuracy: 0.5362
Epoch 8/50


In [ ]:
y_test = np_utils.to_categorical(y_test, numclasses)
	#Evaluate the accuracies
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

7/7 [==============================] - 2s 286ms/step - loss: 0.9362 - accuracy: 0.6727
Test score: 0.9361865520477295
Test accuracy: 0.6726804375648499


In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred1=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
y_pred[0:10]

array([[4.2785510e-02, 8.3952951e-01, 1.1768503e-01],
       [2.8068891e-01, 4.4836950e-01, 2.7094162e-01],
       [3.3815671e-02, 6.5090877e-01, 3.1527555e-01],
       [9.5918030e-04, 8.6644727e-01, 1.3259359e-01],
       [1.2668592e-02, 2.0418102e-01, 7.8315037e-01],
       [1.3427607e-03, 1.1289202e-02, 9.8736805e-01],
       [6.0715312e-01, 5.3344062e-03, 3.8751253e-01],
       [2.5177933e-03, 7.1778603e-02, 9.2570364e-01],
       [9.0782734e-04, 5.7163888e-01, 4.2745328e-01],
       [6.7970355e-04, 9.2619485e-01, 7.3125504e-02]], dtype=float32)

In [ ]:
y_pred1[10:20]

array([2, 2, 2, 0, 1, 0, 2, 1, 1, 1])

In [ ]:
y_test[10:20]

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


ValueError: ignored